In [11]:
import sys
sys.path.append('..')

In [30]:
from main import data_generator
from main import util
import numpy as np
import pandas as pd
import random
from scipy.special import expit

In [76]:
def sample_LY_auto_g(n_samples, network, burn_in, verbose, prob_v_given_boundary_L, prob_v_given_boundary_Y):
    V_DOMAIN = [1, 0]
    samples = []
    num_vertices = len(network)

    for _ in range(n_samples):
        if verbose:
            print("progress: ", _ / n_samples)
            
        sample = pd.DataFrame(index=network.keys(), columns=['L', 'A', 'Y'])
        
        # Initialize with random values
        sample['L'] = {vertex: random.choice(V_DOMAIN) for vertex in network}
        sample['Y'] = {vertex: random.choice(V_DOMAIN) for vertex in network}
        
        # Fill column 'A' with all ones
        sample['A'] = 1
        
        # Use Gibbs sampler to generate 1 sample
        for i in range(burn_in):
            for subject in network.keys():
                # sample L_i
                boundary_values_L = {
                    'L_neighbors': [sample.loc[neighbor, 'L'] for neighbor in network[subject]]
                }
                p_L = prob_v_given_boundary_L(boundary_values_L)
                sample.loc[subject, 'L'] = np.random.choice(V_DOMAIN, size=1, p=np.array([p_L, 1-p_L]))[0]
                
                # sample Y_i
                boundary_values_Y = {
                    'L_self': sample.loc[subject, 'L'],
                    'L_neighbors': [sample.loc[neighbor, 'L'] for neighbor in network[subject]],
                    'A_self': sample.loc[subject, 'A'],
                    'A_neighbors': [sample.loc[neighbor, 'A'] for neighbor in network[subject]],
                    'Y_neighbors': [sample.loc[neighbor, 'Y'] for neighbor in network[subject]],
                }
                p_Y = prob_v_given_boundary_Y(boundary_values_Y)
                sample.loc[subject, 'Y'] = np.random.choice(V_DOMAIN, size=1, p=np.array([p_Y, 1-p_Y]))[0]

        # Add the current sample DataFrame to the list
        samples.append(sample)

    return samples

def sample_LY_ours(n_samples, network, burn_in, verbose, prob_v_given_boundary_L, prob_v_given_boundary_Y):
    V_DOMAIN = [1, 0]
    samples = []
    num_vertices = len(network)
    
    # Use Gibbs sampler to generate the "observed" sample for L
    L_sample = pd.DataFrame(index=network.keys(), columns=['L'])
    L_sample['L'] = {vertex: random.choice(V_DOMAIN) for vertex in network}
    for i in range(burn_in):
        for subject in network.keys():
            # sample L_i
            boundary_values_L = {
                'L_neighbors': [L_sample.loc[neighbor, 'L'] for neighbor in network[subject]]
            }
            p_L = prob_v_given_boundary_L(boundary_values_L)
            L_sample.loc[subject, 'L'] = np.random.choice(V_DOMAIN, size=1, p=np.array([p_L, 1-p_L]))[0]

    for _ in range(n_samples):
        if verbose:
            print("progress: ", _ / n_samples)
        sample = pd.DataFrame(index=network.keys(), columns=['L', 'A', 'Y'])
        
        # Initialize
        sample['L'] = L_sample
        sample['A'] = 1
        sample['Y'] = {vertex: random.choice(V_DOMAIN) for vertex in network}
        
        # Use Gibbs sampler to generate 1 sample of Y
        for i in range(burn_in):
            for subject in network.keys():
                # sample Y_i
                boundary_values_Y = {
                    'L_self': sample.loc[subject, 'L'],
                    'L_neighbors': [sample.loc[neighbor, 'L'] for neighbor in network[subject]],
                    'A_self': sample.loc[subject, 'A'],
                    'A_neighbors': [sample.loc[neighbor, 'A'] for neighbor in network[subject]],
                    'Y_neighbors': [sample.loc[neighbor, 'Y'] for neighbor in network[subject]],
                }
                p_Y = prob_v_given_boundary_Y(boundary_values_Y)
                sample.loc[subject, 'Y'] = np.random.choice(V_DOMAIN, size=1, p=np.array([p_Y, 1-p_Y]))[0]

        # Add the current sample DataFrame to the list
        samples.append(sample)

    return samples

In [77]:
def prob_v_given_boundary_L(boundary_values):
    weighted_sum = 0
    weights = {
        'L_neighbors': 0.2,
    }
    for key, values in boundary_values.items():
        if values is not None:
            if isinstance(values, list):
                weighted_sum += weights[key] * sum(values)
            else:
                weighted_sum += weights[key] * values
    return expit(weighted_sum)

def prob_v_given_boundary_Y(boundary_values):
    weighted_sum = 0
    weights = {
        'Y_neighbors': 0.4,
        'L_self': -2.0,
        'A_self': -1,
        'L_neighbors': 0.4,
        'A_neighbors': 0.4
    }
    for key, values in boundary_values.items():
        if values is not None:
            if isinstance(values, list):
                weighted_sum += weights[key] * sum(values)
            else:
                weighted_sum += weights[key] * values
    return expit(weighted_sum)

In [ ]:
NUM_OF_VERTICES = 10
VERBOSE = True
BURN_IN = 20
N_SAMPLES = 1000

network = util.create_random_network(n=NUM_OF_VERTICES, min_neighbors=1, max_neighbors=3)
auto_g_1 = sample_LY_auto_g(N_SAMPLES, network, BURN_IN, VERBOSE, prob_v_given_boundary_L, prob_v_given_boundary_Y)
auto_g_2 = sample_LY_auto_g(N_SAMPLES, network, BURN_IN, VERBOSE, prob_v_given_boundary_L, prob_v_given_boundary_Y)
ours = sample_LY_ours(N_SAMPLES, network, BURN_IN, VERBOSE, prob_v_given_boundary_L, prob_v_given_boundary_Y)

progress:  0.0
progress:  0.001
progress:  0.002
progress:  0.003
progress:  0.004
progress:  0.005
progress:  0.006
progress:  0.007
progress:  0.008
progress:  0.009
progress:  0.01
progress:  0.011
progress:  0.012
progress:  0.013
progress:  0.014
progress:  0.015
progress:  0.016
progress:  0.017
progress:  0.018
progress:  0.019
progress:  0.02
progress:  0.021
progress:  0.022
progress:  0.023
progress:  0.024
progress:  0.025
progress:  0.026
progress:  0.027
progress:  0.028
progress:  0.029
progress:  0.03
progress:  0.031
progress:  0.032
progress:  0.033
progress:  0.034
progress:  0.035
progress:  0.036
progress:  0.037
progress:  0.038
progress:  0.039
progress:  0.04
progress:  0.041
progress:  0.042
progress:  0.043
progress:  0.044
progress:  0.045
progress:  0.046
progress:  0.047
progress:  0.048
progress:  0.049
progress:  0.05
progress:  0.051
progress:  0.052
progress:  0.053
progress:  0.054
progress:  0.055
progress:  0.056
progress:  0.057
progress:  0.058
prog

progress:  0.486
progress:  0.487
progress:  0.488
progress:  0.489
progress:  0.49
progress:  0.491
progress:  0.492
progress:  0.493
progress:  0.494
progress:  0.495
progress:  0.496
progress:  0.497
progress:  0.498
progress:  0.499
progress:  0.5
progress:  0.501
progress:  0.502
progress:  0.503
progress:  0.504
progress:  0.505
progress:  0.506
progress:  0.507
progress:  0.508
progress:  0.509
progress:  0.51
progress:  0.511
progress:  0.512
progress:  0.513
progress:  0.514
progress:  0.515
progress:  0.516
progress:  0.517
progress:  0.518
progress:  0.519
progress:  0.52
progress:  0.521
progress:  0.522
progress:  0.523
progress:  0.524
progress:  0.525
progress:  0.526
progress:  0.527
progress:  0.528
progress:  0.529
progress:  0.53
progress:  0.531
progress:  0.532
progress:  0.533
progress:  0.534
progress:  0.535
progress:  0.536
progress:  0.537
progress:  0.538
progress:  0.539
progress:  0.54
progress:  0.541
progress:  0.542
progress:  0.543
progress:  0.544
prog

progress:  0.972
progress:  0.973
progress:  0.974
progress:  0.975
progress:  0.976
progress:  0.977
progress:  0.978
progress:  0.979
progress:  0.98
progress:  0.981
progress:  0.982
progress:  0.983
progress:  0.984
progress:  0.985
progress:  0.986
progress:  0.987
progress:  0.988
progress:  0.989
progress:  0.99
progress:  0.991
progress:  0.992
progress:  0.993
progress:  0.994
progress:  0.995
progress:  0.996
progress:  0.997
progress:  0.998
progress:  0.999
progress:  0.0
progress:  0.001
progress:  0.002
progress:  0.003
progress:  0.004
progress:  0.005
progress:  0.006
progress:  0.007
progress:  0.008
progress:  0.009
progress:  0.01
progress:  0.011
progress:  0.012
progress:  0.013
progress:  0.014
progress:  0.015
progress:  0.016
progress:  0.017
progress:  0.018
progress:  0.019
progress:  0.02
progress:  0.021
progress:  0.022
progress:  0.023
progress:  0.024
progress:  0.025
progress:  0.026
progress:  0.027
progress:  0.028
progress:  0.029
progress:  0.03
prog

progress:  0.458
progress:  0.459
progress:  0.46
progress:  0.461
progress:  0.462
progress:  0.463
progress:  0.464
progress:  0.465
progress:  0.466
progress:  0.467
progress:  0.468
progress:  0.469
progress:  0.47
progress:  0.471
progress:  0.472
progress:  0.473
progress:  0.474
progress:  0.475
progress:  0.476
progress:  0.477
progress:  0.478
progress:  0.479
progress:  0.48
progress:  0.481
progress:  0.482
progress:  0.483
progress:  0.484
progress:  0.485
progress:  0.486
progress:  0.487
progress:  0.488
progress:  0.489
progress:  0.49
progress:  0.491
progress:  0.492
progress:  0.493
progress:  0.494
progress:  0.495
progress:  0.496
progress:  0.497
progress:  0.498
progress:  0.499
progress:  0.5
progress:  0.501
progress:  0.502
progress:  0.503
progress:  0.504
progress:  0.505
progress:  0.506
progress:  0.507
progress:  0.508
progress:  0.509
progress:  0.51
progress:  0.511
progress:  0.512
progress:  0.513
progress:  0.514
progress:  0.515
progress:  0.516
prog

progress:  0.944
progress:  0.945
progress:  0.946
progress:  0.947
progress:  0.948
progress:  0.949
progress:  0.95
progress:  0.951
progress:  0.952
progress:  0.953
progress:  0.954
progress:  0.955
progress:  0.956
progress:  0.957
progress:  0.958
progress:  0.959
progress:  0.96
progress:  0.961
progress:  0.962
progress:  0.963
progress:  0.964
progress:  0.965
progress:  0.966
progress:  0.967
progress:  0.968
progress:  0.969
progress:  0.97
progress:  0.971
progress:  0.972
progress:  0.973
progress:  0.974
progress:  0.975
progress:  0.976
progress:  0.977
progress:  0.978
progress:  0.979
progress:  0.98
progress:  0.981
progress:  0.982
progress:  0.983
progress:  0.984
progress:  0.985
progress:  0.986
progress:  0.987
progress:  0.988
progress:  0.989
progress:  0.99
progress:  0.991
progress:  0.992
progress:  0.993
progress:  0.994
progress:  0.995
progress:  0.996
progress:  0.997
progress:  0.998
progress:  0.999
progress:  0.0
progress:  0.001
progress:  0.002
prog

In [83]:
def test_dist(df1, df2):

    diff = []

    for subject in range(NUM_OF_VERTICES):
        df1_value_of_subject = []
        df2_value_of_subject = []

        for sample in range(N_SAMPLES):
            df1_value_of_subject.append(df1[sample]['Y'][subject])
            df2_value_of_subject.append(df2[sample]['Y'][subject])

        diff.append(sum(df1_value_of_subject) / N_SAMPLES - sum(df2_value_of_subject) / N_SAMPLES)

    print(np.mean(diff))

0.007799999999999974

In [80]:
diff

[0.16999999999999993,
 0.09999999999999998,
 -0.26999999999999996,
 0.08000000000000002,
 0.0,
 -0.040000000000000036,
 -0.14,
 -0.030000000000000027,
 -0.020000000000000018,
 0.11000000000000004]

In [84]:
diff

[0.22999999999999998,
 0.07299999999999995,
 -0.22100000000000003,
 -0.19100000000000006,
 -0.29799999999999993,
 0.219,
 -0.29700000000000004,
 0.14499999999999996,
 0.22699999999999998,
 0.19099999999999995]